In [1]:
import keras
import numpy as np
from keras.utils import to_categorical
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 

Using TensorFlow backend.


In [2]:
def setting():
    info = {
        'data_path' : "dataset_training/video_phases_background_removed.pickle"
    }
    return info

def loaddata(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data

def vector_cat(label_name, le):
    vec = np.zeros(len(list(le.classes_)))
    vec[int(le.transform([label_name]))] = 1
    return vec

def set_x_y(data):
    
    labels = list(data.keys())
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    x = []
    y = []
    for key in data.keys():
        for img in data[key]:
            x.append(img)
            y.append(vector_cat(key, le))
    return x,y, le
## Setting
info = setting()
data = loaddata(info['data_path'])
x, y, le = set_x_y(data)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.33)
train_x=np.asarray(train_x)
test_x=np.asarray(test_x)
train_y=np.asarray(train_y)
test_y=np.asarray(test_y)

train_x = train_x.reshape(-1, 1,200, 230)/255
test_x = test_x.reshape(-1, 1,200, 230)/255

In [3]:
# main
## Learning
model = Sequential()
model.add(Convolution2D(batch_input_shape=(None, 1, 200, 230),filters=32,kernel_size=5,strides=1,padding='same', data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same',data_format='channels_first'))
model.add(Convolution2D(64, 5, strides=1, padding='same',data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))
model.add(Convolution2D(64, 5, strides=1, padding='same',data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))
model.add(Convolution2D(64, 5, strides=1, padding='same',data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))
model.add(Convolution2D(64, 5, strides=1, padding='same',data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))
model.add(Flatten())
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(len(le.classes_)))
model.add(Activation('softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
print('Training----------')
model.fit(train_x, train_y, epochs=150, batch_size=100)
# evaluate the model
print('\nTesting----------')
loss, accuracy = model.evaluate(test_x, test_y)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

Training----------
Epoch 1/150
7562/7562 [==============================] - 16s 2ms/step - loss: 0.7416 - acc: 0.6912
Epoch 2/150
7562/7562 [==============================] - 11s 2ms/step - loss: 0.2700 - acc: 0.8939
Epoch 3/150
7562/7562 [==============================] - 11s 2ms/step - loss: 0.2127 - acc: 0.9111
Epoch 4/150
7562/7562 [==============================] - 11s 2ms/step - loss: 0.1861 - acc: 0.9267
Epoch 5/150
7562/7562 [==============================] - 12s 2ms/step - loss: 0.1415 - acc: 0.9390
Epoch 6/150
7562/7562 [==============================] - 12s 2ms/step - loss: 0.1107 - acc: 0.9508
Epoch 7/150
7562/7562 [==============================] - 12s 2ms/step - loss: 0.1004 - acc: 0.9583
Epoch 8/150
7562/7562 [==============================] - 12s 2ms/step - loss: 0.0947 - acc: 0.9550
Epoch 9/150
7562/7562 [==============================] - 12s 2ms/step - loss: 0.0928 - acc: 0.9562
Epoch 10/150
7562/7562 [==============================] - 12s 2ms/step - loss: 0.0812 - ac

KeyboardInterrupt: 

In [13]:

# evaluate the model
print('\nTesting----------')
loss, accuracy = model.evaluate(test_x, test_y)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)


Testing----------
3726/3726 [==============================] - 3s 841us/step

test loss:  0.1131247414962873

test accuracy:  0.954106280097255


In [12]:
model.save('seq_acu_95.h5')

In [10]:
np.argmax(model.predict(np.array([test_x[1]])))

4

In [11]:
test_y[1]

array([0., 0., 0., 0., 1.])

In [15]:
with open('seq_acu_95_train_x.pickle','wb') as file:
    pickle.dump(train_x, file)
with open('seq_acu_95_train_y.pickle','wb') as file:
    pickle.dump(train_y, file)
with open('seq_acu_95_test_x.pickle','wb') as file:
    pickle.dump(test_x, file)
with open('seq_acu_95_test_y.pickle','wb') as file:
    pickle.dump(test_y, file)
    
with open('seq_acu_95_encoder.pickle','wb') as file:
    pickle.dump(le, file)


# Reference
1. [莫煩的 CNN example](https://github.com/MorvanZhou/tutorials/blob/master/kerasTUT/6-CNN_example.py)
2. [莫煩的 CNN 解說](https://morvanzhou.github.io/tutorials/machine-learning/keras/2-3-CNN/)
3. [GPU 設定](https://blog.csdn.net/u010420283/article/details/85261633)
# Trial and error
- 如果 Maxpooling 太少, 會導致記憶體不夠
    - Maxpooling 會維持二維
    - Maxpooling 交替 covlution2D
    - 持續增加 maxpooling, 方可解決 

# Procedure
- Encode
    - OneHotEncoding
- Training
    - Maxpooling 會維持二維
    - Maxpooling 交替 covlution2D
    - Flatten
    - Dense
    - Output